In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
import tensorflow
import skimage
import skimage.io
import skimage.transform
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.layers import Dropout, BatchNormalization,LeakyReLU, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the saved model
from keras.models import model_from_json

In [2]:
np.random.seed(11)
tensorflow.random.set_seed(11)

In [4]:
img_folder='C:\\Users\\satvi\\OneDrive\\Pictures\\Desktop\\Test1\\static2\\images\\'
img_width=128
#chck path
img_height=128
img_channels=3
bees=pd.read_csv('C:\\Users\\satvi\\OneDrive\\Pictures\\Desktop\\Test1\\beehive_data.csv', 
            
                index_col=False,  
                parse_dates={'datetime':[1,2]},
                dtype={'subspecies':'category', 'health':'category'})
def read_img(file):
    img = skimage.io.imread(img_folder + file)
    img = skimage.transform.resize(img, (img_width, img_height), mode='reflect')
    if img.shape[2] == 4:
        img = img[:, :, :3]
    return img
    # return img[:,:,:img_channels]

bees.dropna(inplace=True)
img_exists = bees['file'].apply(lambda f: os.path.exists(img_folder + f))
bees = bees[img_exists]

In [5]:
bees.head()

,datetime,file,location,zip code,subspecies,health
0,2024-01-21 14:00:00,01_001.png,"Kalakadu, Tamil Nadu",627007,Dammer beehive,Healthy
1,2024-01-22 15:00:00,01_002.png,"Kalakadu, Tamil Nadu",627007,Dammer beehive,Healthy
2,2024-01-23 16:00:00,01_003.png,"Kalakadu, Tamil Nadu",627007,Dammer beehive,Healthy
3,2024-01-24 17:00:00,01_004.png,"Kalakadu, Tamil Nadu",627007,Dammer beehive,Healthy
4,2024-01-25 18:00:00,01_005.png,"Kalakadu, Tamil Nadu",627007,Dammer beehive,Healthy


In [6]:
bees.tail()

,datetime,file,location,zip code,subspecies,health
102,2024-01-24 10:00:00,04_028.png,"Bela Gerya, West Bengal",721126,Indian hive beehive,Healthy
103,2024-01-24 10:00:00,04_029.png,"Bela Gerya, West Bengal",721126,Indian hive beehive,Healthy
104,2024-01-24 10:00:00,04_030.png,"Bela Gerya, West Bengal",721126,Indian hive beehive,Healthy
105,2024-01-24 10:00:00,04_031.png,"Bela Gerya, West Bengal",721126,Indian hive beehive,Unhealthy
106,2024-01-24 10:00:00,04_032.png,"Bela Gerya, West Bengal",721126,Indian hive beehive,Unhealthy


In [7]:
bees.isna()

,datetime,file,location,zip code,subspecies,health
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
102,False,False,False,False,False,False
103,False,False,False,False,False,False
104,False,False,False,False,False,False
105,False,False,False,False,False,False


In [8]:
train_bees, test_bees = train_test_split(bees, random_state=65)
train_bees, val_bees = train_test_split(train_bees, test_size=0.1, random_state = 67)
ncat_bal = int(len(train_bees)/train_bees['health'].cat.categories.size)
train_bees_bal = train_bees.groupby('health', as_index=False).apply(lambda g:  g.sample(ncat_bal, replace=True)).reset_index(drop=True)
train_bees = train_bees_bal


In [9]:
train_X = np.stack(train_bees['file'].apply(read_img))
train_y  = pd.get_dummies(train_bees['health'], drop_first=False)

val_X = np.stack(val_bees['file'].apply(read_img))
val_y = pd.get_dummies(val_bees['health'], drop_first=False)

test_X = np.stack(test_bees['file'].apply(read_img))
test_y = pd.get_dummies(test_bees['health'], drop_first=False)

# Data augmentation - a little bit rotate, zoom and shift input images.
generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)
generator.fit(train_X)

In [10]:
earlystopper = EarlyStopping(monitor='val_accuracy', patience=25, verbose=1)
checkpointer = ModelCheckpoint('beehive_health.h5'
                                ,monitor='val_accuracy'
                                ,verbose=1
                                ,save_best_only=True
                                ,save_weights_only=True)
model=Sequential()
model.add(Conv2D(32, kernel_size=3, input_shape=(img_width, img_height,3), activation='relu', padding='same'))
model.add(MaxPool2D(2))
model.add(Dropout(0.15))
model.add(Conv2D(32, kernel_size=3, activation='relu', padding='same'))
model.add(Dropout(0.15))
model.add(Flatten())
model.add(Dense(train_y.columns.size, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

batch_size = 32  # Adjust as needed

# Calculate steps_per_epoch
steps_per_epoch = len(train_X) // batch_size
if len(train_X) % batch_size != 0:
    steps_per_epoch += 1

# Adjust generator flow and fit_generator
training = model.fit_generator(
    generator.flow(train_X, train_y, batch_size=batch_size),
    epochs=200,
    validation_data=(val_X, val_y),
    steps_per_epoch=steps_per_epoch,
    callbacks=[earlystopper, checkpointer]
)
# steps = np.round(train_X.shape[0] / 256, 0)
# training = model.fit_generator(generator.flow(train_X,train_y, batch_size=256)
#                         ,epochs=200
#                         ,validation_data=(val_X, val_y)
#                         ,steps_per_epoch=steps
#                         ,callbacks=[earlystopper, checkpointer])

C:\Users\satvi\AppData\Local\Temp\ipykernel_11088\4042361497.py:25: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  training = model.fit_generator(


Epoch 1/200
3/3 [==============================] - ETA: 0s - loss: 3.4508 - accuracy: 0.4722
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to beehive_health.h5
3/3 [==============================] - 8s 2s/step - loss: 3.4508 - accuracy: 0.4722 - val_loss: 2.7246 - val_accuracy: 0.0000e+00
Epoch 2/200
3/3 [==============================] - ETA: 0s - loss: 1.6931 - accuracy: 0.5000
Epoch 2: val_accuracy did not improve from 0.00000
3/3 [==============================] - 3s 1s/step - loss: 1.6931 - accuracy: 0.5000 - val_loss: 1.4326 - val_accuracy: 0.0000e+00
Epoch 3/200
3/3 [==============================] - ETA: 0s - loss: 0.7104 - accuracy: 0.5000
Epoch 3: val_accuracy improved from 0.00000 to 1.00000, saving model to beehive_health.h5
3/3 [==============================] - 1s 576ms/step - loss: 0.7104 - accuracy: 0.5000 - val_loss: 0.4977 - val_accuracy: 1.0000
Epoch 4/200
3/3 [==============================] - ETA: 0s - loss: 0.7128 - accuracy: 0.5000
Epoch 4: va

In [11]:
# model.load_weights('C:\\Users\\satvi\\OneDrive\\Desktop\\Bee(current 2)\\best.h5')
# model_json = model.to_json()
# with open("model_architecture.json", "w") as json_file:
#     json_file.write(model_json)
model_json = model.to_json()
with open("Hive_health.json", "w") as json_file:
    json_file.write(model_json)

# Save model weights
model.save_weights('beehive_health.h5')
test_res = model.evaluate(test_X, test_y.values, verbose=0)
predictions = model.predict(test_X)

# Convert one-hot encoded predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_y.values, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

1/1 [==============================] - 1s 741ms/step
Accuracy: 0.6296296296296297
Precision: 0.4722222222222222
Recall: 0.3269230769230769
F1-score: 0.38636363636363635
